In [3]:
import pandas as pd
import numpy as np
import re

1. value: Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $
2. exportValue: Value of actual exports based on five-year averages of direct and mirror reports
3. realizedPotential: C/B if C<B, 1 if C>B
4. lineLength: Projected market share (corrected for global tariff advantages, and subject to a re-export filter)
5. bubbleSize: Projected import value (accounting for product-specific tariff and distance advantages)
6. lineWidth: Ease of trade with the target market, independent of size or complementarity in trade structure
7. gap: Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth
8. rank: Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $

In [4]:
df3 = pd.read_excel('document_3.xlsx')
df3.head(2)

,product code,"value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)",exportValue (Value of actual exports based on five-year averages of direct and mirror reports),"realizedPotential (C/B if C<B, 1 if C>B)","lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))",bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages)),"lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)",tariff,xik,mjk,xij,"gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)","rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)",from,to,code,name,color,parent
0,1001Xb,3.248636e+07,358000.0,0.01102,161.860859,5.617304e+07,1.812191,0.025,6.159614e+09,1.000327e+08,1.467541e+08,0.98898,1,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
1,151219,2.237364e+07,44721000.0,1.00000,113.379827,6.956521e+07,1.812191,0.025,4.942541e+08,5.824338e+07,1.467541e+08,0.00000,2,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."


In [5]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392542 entries, 0 to 392541
Data columns (total 19 columns):
 #   Column                                                                                                                                               Non-Null Count   Dtype  
---  ------                                                                                                                                               --------------   -----  
 0   product code                                                                                                                                         392542 non-null  object 
 1   value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)  392542 non-null  float64
 2   exportValue (Value of actual exports based on five-year averages of direct and mirror reports)                                                       392542 non-null  fl

In [6]:
df3.columns

Index(['product code',
       'value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)',
       'exportValue (Value of actual exports based on five-year averages of direct and mirror reports)',
       'realizedPotential (C/B if C<B, 1 if C>B)',
       'lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))',
       'bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages))',
       'lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)',
       'tariff', 'xik', 'mjk', 'xij',
       'gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)',
       'rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, express

In [7]:
df3.rename(columns={'value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)': 'value', 'exportValue (Value of actual exports based on five-year averages of direct and mirror reports)':'exportValue', 'realizedPotential (C/B if C<B, 1 if C>B)': 'realizedPotential', 'lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))': 'lineLength', 'bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages))': 'bubbleSize', 'lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)': 'lineWidth', 'gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)': 'gap', 'rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)': 'rank'}, inplace=True)
df3 = df3.drop(['xik', 'mjk', 'xij'], axis=1)
df3.columns


Index(['product code', 'value', 'exportValue', 'realizedPotential',
       'lineLength', 'bubbleSize', 'lineWidth', 'tariff', 'gap', 'rank',
       'from', 'to', 'code', 'name', 'color', 'parent'],
      dtype='object')

In [8]:
df3.head(2)

,product code,value,exportValue,realizedPotential,lineLength,bubbleSize,lineWidth,tariff,gap,rank,from,to,code,name,color,parent
0,1001Xb,3.248636e+07,358000.0,0.01102,161.860859,5.617304e+07,1.812191,0.025,0.98898,1,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
1,151219,2.237364e+07,44721000.0,1.00000,113.379827,6.956521e+07,1.812191,0.025,0.00000,2,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
